In [0]:
import json
import os
import pandas as pd
import csv
import keras
from numpy import array
import matplotlib.pyplot as plt

In [0]:
# the bodylandmark directory for a vedio
video_landmark_path = '/content/drive/My Drive/636project/data/adl_UR/adl-20-cam0'

data = []

# read json 2-d pose as data, without using: 
#     {15, "REye"},
#     {16, "LEye"},
#     {17, "REar"},
#     {18, "LEar"}
frames = os.listdir(video_landmark_path)

for frame in frames:
  frame_path = os.path.join(video_landmark_path, frame)
  with open(frame_path, mode='r') as json_file:
    people_dict = json.load(json_file)
    people = people_dict["people"]
    pose_keypoints_2d = []
    # fill missing data as 0
    if len(people) == 0:
      pose_keypoints_2d = [0] * 63
    else:
      full_pose = people[0].get("pose_keypoints_2d")
      pose_keypoints_2d = full_pose[:45] + full_pose[57:]
    # each pose_keypoints_2d: 
    # [431.949, 196.241, 0.0564434, 437.194, 187.749, 0.552267,...]
    frame_number = int(frame.split('.')[0].split('_')[1])
    pose_keypoints_2d.append(frame_number)
    data.append(pose_keypoints_2d) 

data = sorted(data, key=lambda l:l[-1])

for d in data:
  d.remove(d[-1])

df = pd.DataFrame.from_records(data)
df.to_csv (r'/content/drive/My Drive/636project/data.csv', index = False, header=False)

In [0]:
# use model and write the json file for a video
print("loading model and computing probability for each frame...")
model = keras.models.load_model('/content/drive/My Drive/636project/model.h5')
dictionary = {}
dictionary['falling'] = []
test = []

# normalize data 
with open('/content/drive/My Drive/636project/data.csv', mode='r') as csv_file:
  reader = csv.reader(csv_file)
  for record in reader:
    for i in range(0, 63, 3):
      record[i] = float(record[i])/ 640
      record[i + 1] = float(record[i + 1]) / 480
    test.append(record)

test = array(test)
test = test.reshape((len(test), 1, len(test[0])))
probality = model.predict_proba(test)

frame_num = 0
for i in range(len(probality)):
    timestamp = frame_num / 30
    probability_fall = float(probality[i][0])
    to_add = [timestamp, probability_fall]
    dictionary['falling'].append(to_add)
    frame_num +=1

# Serializing json  
json_object = json.dumps(dictionary) 
  
print("complete computing, and the result has been written to json file")
# Writing to sample.json 
with open("/content/drive/My Drive/636project/result.json", "w") as outfile: 
    outfile.write(json_object) 

In [0]:
df = pd.DataFrame(dictionary['heads up'])
df.plot(x=0, y=1)
plt.savefig('/content/drive/My Drive/636project/output_plot.png')